# 데이터 전처리

In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('vgames2.csv'); df = df[df.columns[1:]]
df.head(3)

,Name,Platform,Year,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales
0,Candace Kane's Candy Factory,DS,2008.0,Action,Destineer,0.04,0,0,0
1,The Munchables,Wii,2009.0,Action,Namco Bandai Games,0.17,0,0,0.01
2,Otome wa Oanesama Boku ni Koi Shiteru Portable,PSP,2010.0,Adventure,Alchemist,0,0,0.02,0


## 중복 값 제거 (16,598 -> 16,597)
> 완전 중복 값만 제거하고,<br>
> 동일 게임 명이지만 플랫폼이 다른 등의 중복 값은 합치기 등의 처리는 배제함

In [3]:
df = df.drop_duplicates() # 1개 삭제

## Nan Data 제거 (16,597 -> 16,241)
> Nan Data 비중이 2%(356/16597)로 조금의 비중은 있게 판단되나 <br>
> 대체 값을 넣기에는 오류 데이터를 만들 가능성이 있기에 <br>
> 분석 불가능 데이터로 규정하고 배제로 결정함

In [4]:
df = df.dropna() # 356(year=270, genre=50, publisher=58 , 중복 존재)개 삭제

## 각 칼럼 조정 (16,241 -> 12,303)
*  수치 데이터(연도, 출고량)는 Int로 수식 연산이 가능하도록 함
*  글자 데이터(이름, 플랫폼, 장르, 제작사)는 Oject로 비교 연산 중점으로 전처리

<br>

*  **중점** 데이터는 **수치데이터**와 **플랫폼, 장르** 데이터로 규정하고 <br>
   해당 데이터들을 중심으로 전처리를 진행하고, 수식 연산이 가능하도록 한다.

*  이름과 제작사는 결과 값 Index와 같으므로 중복허용&거의 전처리 없이 진행하되
   <br>결과 값에 대한 이름과 제작사의 확인 여부를 후속 검증하도록 한다.

> 플랫폼열부터 값이 당겨진 데이터는 소수(2개)임으로 사용자가 원본파일에서 수정함

### 1.) **Year(연도)** : 자료형은 Int로 대소, 범위 연산 진행을 할 수 있게 한다
> 100 이하인 데이터 변경 ( 0 ~ 16 = 2000년 , 86 ~ 98 = 1900년) <br>
> \>> 사용자의 일괄 조정이므로 오류 데이터를 만들 수 있지만 271개의 Nan 소실로 <br>
> 'Year'로 인한 추가적인 100개의 손실 방지를 위해 해당 조정을 진행함

In [5]:
df.Year = df.Year.astype(int)
under20, upper85 = df[df.Year < 20] , df[ ( (df.Year > 85) & (df.Year < 100) )]
df.Year[under20.index] += 2000; df.Year[upper85.index] += 1900;

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


### 2) **Genre(장르)** : 자료형은 Oject로 등호 연산 진행 / (16,241 -> 13,687)
> 오류 2종 제거(Misc(기타, 1,681개) 와 Platform(오류 데이터 기재, 873개) )

> Misc(기타) 값은 추후 장르 분석에서 명확한 구분이 되지 않는 데이터로 제거함<br>
> Platform 값은 플랫폼 열에는 값이 있기에 잘못 기재한 오류데이터로 제거함

In [6]:
df = df[ (df['Genre'] != 'Misc') & (df['Genre'] != 'Platform') ] # 총 2,554개

### 3) **Platform(플랫폼)** : 자료형은 Oject로 등호 연산 진행 / (13,687 -> 11,240)
> 오류 값 제거('2600'): 각 행에 대체 값이 없고, 연도치곤 수치가 너무 커서 제거함

> 이하의 10개 플랫폼은 자료의 내용에 따라 데이터 분석에서 제외함 <br>
> 이유: 특정 국가에서만 판매 혹은 생산량 100개 이하(개발 시 참고 데이터 부족) 
> <br> 혹은 최근 10년 이내로 생산되지 않아 단종으로 취급하여 제외함

> 'PCFX'은 1996년 1회 생산(일본에서만 판매) 제외함 <br>
> 'TG16'은 1995년 2회 생산(일본에서만 판매) 제외함 <br>
> '3DO'은 1994년과 5년 총 3회 생산(일본에서만 판매) 제외함 <br>
> 'SCD'은 1994년 3회 생산(일본에서만 판매) 제외함 <br>
> 'WS'은 1999 ~ 2001년 총 6회 생산(일본에서만 판매) 제외함 <br>
> 'NG'은 1993 ~ 6년 총 12회 생산(일본에서만 판매) 제외함 <br>
> 'GEN'은 1990 ~ 6년 총 19회 생산으로 제외함 <br>
> 'DC'은 1998 ~ 2008년 총 50회 생산으로 제외함 <br>
> 'NES'은 1983 ~ 94년 총 68회 생산으로 제외함 <br>
> 'GB'은 1989 ~ 2001년 총 71회 생산으로 제외함

> 'SNES' & 'SAT'는 생산이 100건 이상이지만 자료상 1999년 이후 단종으로 제외함

> 'GC'와 'N64' 플랫폼은 각각 2000, 2003년에 생산 종료라는 도메인으로 제외함

> 'Wii'는 13년에 단종, 19년에 소프트 구입 불가능하다는 도메인으로 제외함

In [7]:
df = df[ df['Platform'] != '2600' ] # 101개 데이터

In [8]:
df = df[ (df['Platform'] != 'PCFX') & (df['Platform'] != 'TG16') & (df['Platform'] != '3DO') & (df['Platform'] != 'SCD') & (df['Platform'] != 'WS') &
         (df['Platform'] != 'GEN') & (df['Platform'] != 'DC') & (df['Platform'] != 'NES') & (df['Platform'] != 'GB') & (df['Platform'] != 'NG') &
         (df['Platform'] != 'SNES') & (df['Platform'] != 'SAT') &
         (df['Platform'] != 'GC') & (df['Platform'] != 'N64') &
         (df['Platform'] != 'Wii') & (df['Platform'] != 'WiiU') ]

### 4) Name(이름) : 자료형은 Oject로 등호 연산 진행을 할 수 있게 한다
> 오류 제거(.hack//오류만 제거, 나머지는 도메인 부족으로 후속검증 조치)

In [9]:
df.Name = df.Name.str.replace('.hack//',''); # 8개 데이터

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  """Entry point for launching an IPython kernel.


### 5) **Sales(출고량)** : 자료형은 Int로 대소, 비교 연산 진행을 할 수 있게 한다
> K,M 문자 제거, 전체 100을 곱하고, K는 10을 , M은 10,000을 추가로 곱한다 <br>
> Int형 연산 및 해석의 용의를 위해 최솟값을 1(개)로 지정한다. <br>
<br>
> 개별 출고량들을 합한 ['전체 출고량'] 열을 추가한다. <br>
> 플랫폼을 수치화하여 ['Platform_num'] 열을 추가한다.

In [10]:
bm = df.NA_Sales.str.extract('(\d+M$)').dropna() # 북미 출고량 조정
bk = df.NA_Sales.str.extract('(\d+K$)').dropna()
df.NA_Sales = df.NA_Sales.str.replace('K','').str.replace('M','').astype(float);
df.NA_Sales *= 100; df.NA_Sales[bk.index] *= 10; df.NA_Sales[bm.index] *= 10000;
df.NA_Sales = df.NA_Sales.astype(int);

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [11]:
cm = df.EU_Sales.str.extract('(\d+M$)').dropna() # EU 출고량 조정
ck = df.EU_Sales.str.extract('(\d+K$)').dropna()
df.EU_Sales = df.EU_Sales.str.replace('K','').str.replace('M','').astype(float)
df.EU_Sales *= 100; df.EU_Sales[ck.index] *= 10; df.EU_Sales[cm.index] *= 10000;
df.EU_Sales = df.EU_Sales.astype(int)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [12]:
dm = df.JP_Sales.str.extract('(\d+M$)').dropna() # 일본 출고량 조정
dk = df.JP_Sales.str.extract('(\d+K$)').dropna()
df.JP_Sales = df.JP_Sales.str.replace('K','').str.replace('M','').astype(float)
df.JP_Sales *= 1000; df.JP_Sales[dk.index] *= 10; df.JP_Sales[dm.index] *= 10000;
df.JP_Sales = df.JP_Sales.astype(int)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [13]:
em = df.Other_Sales.str.extract('(\d+M$)').dropna() # 기타 출고량 조정
ek = df.Other_Sales.str.extract('(\d+K$)').dropna()
df.Other_Sales = df.Other_Sales.str.replace('K','').str.replace('M','').astype(float)
df.Other_Sales *= 1000; df.Other_Sales[ek.index] *= 10; df.Other_Sales[em.index] *= 10000;
df.Other_Sales = df.Other_Sales.astype(int)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [14]:
df['Total_Sales'] = df.NA_Sales + df.EU_Sales + df.JP_Sales + df.Other_Sales

In [15]:
a , b = df.Genre.value_counts().index , df.Platform.value_counts().index; rea , reb = {} , {}; list1 , list2 = df.Genre , df.Platform
for i in range(len(b)): # 플랫폼 수치화
  emp = b[i]; reb[emp]=i+1

for key , value in reb.items(): # ['Platform_num'] 열을 추가
  list2 = list2.replace(key, value)

df['Platform_num'] = list2

## 데이터 전처리 종료
> 기본 전처리가 종료된 11,240개의 데이터를 모집단으로 재설정한다.



In [18]:
df.to_csv('전처리완료.csv', index=False)

# 데이터 정의
## 중점 분석 데이터 - 수치(연도&판매량)와 플랫폼&장르 데이터

### **Year(연도)** : 1985 ~ 2020년
> 연도로써 최근 10년의 의미는 2006~2016년으로 정의함 <br>

> 이유: 2020, 2017년을 분석에 포함하기에는 각각 1, 3개의 데이터로 빈약하고 <br>
> 2019 ~ 2018년의 데이터가 없기에 2016년부터를 기준으로 함

### **Salse - each & Total**
출고량은 소비자의 구매량이므로 출고량 0은 무료게임이 아닌 판매가 안 된 제품임 <br> 출고량 1의 의미는 1개라는 의미로 해석함

### **Platform(플랫폼)** 남은 13개 세부 그룹화  <BR>
1) 플레이스테이션 시리즈(본체 650,000원) : PS, PS2, PS3&PSP , PS4&PSV <BR>
2) X-Box 시리즈(본체 630,000원) : XB, X360, Xone <BR>
3) 닌텐도 시리즈(본체 380,000원) : DS , 3DS <BR>
4) 기타 : PC , GBA

## Index적 역활 데이터 - 이름과 제작사
분석을 통한 결과 값(1~3개, 소수)이 나오면 사후 검증으로 조사함